It is just an shortcut for me. These are the functions that I generally use in EDA step. I didn't write any explanation or how to interpret them. You can also benefit from this notebook. If you have a problem, feel free to make a comment.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore', category = FutureWarning)

sns.set_style("darkgrid")

In [ ]:
titanic = pd.read_csv("../input/titanic/train.csv") #Classification
house_prices = pd.read_csv("../input/house-prices-advanced-regression-techniques/train.csv") #regression
car_price = pd.read_csv("../input/car-price-prediction/CarPrice_Assignment.csv") #regression
mushroom = pd.read_csv("../input/mushroom-classification/mushrooms.csv") #classification
mobile = pd.read_csv("../input/mobile-price-classification/train.csv") #classification

# 1) Categorical Variable vs Categorical Target

In [ ]:
def count_percentage(df, col, hue):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(22, 6))
    order = sorted(df[col].unique())
    
    sns.countplot(col, data = df, hue = hue, ax = ax1, order = order).set_title("Counts For Feature:\n" + col)

    df_temp = df.groupby(col)[hue].value_counts(normalize = True).\
    rename("percentage").\
    reset_index()

    fig = sns.barplot(x = col, y = "percentage", hue = hue, data = df_temp, ax = ax2, order = order)
    fig.set_ylim(0,1)
    
    fontsize = 14 if len(order) <= 10 else 10
    for p in fig.patches:
        txt = str(p.get_height().round(2)) + '%'
        txt_x = p.get_x() 
        txt_y = p.get_height()
        fig.text(txt_x + 0.125, txt_y + 0.02,txt, fontsize = fontsize)

    ax2.set_title("Percentages For Feature: \n" + col)

mixed up when we have high cardinality cat. feature

In [ ]:
count_percentage(titanic, "Sex", "Survived")

In [ ]:
count_percentage(mushroom, "gill-color", "class")

In [ ]:
count_percentage(house_prices, "MSSubClass", "MSZoning")

# 2) Numerical Variable vs Categorical Target

In [ ]:
def feature_dist_clas(df, col, hue):
    fig, axes = plt.subplots(1, 4, figsize = (25, 5))
    order = sorted(df[hue].unique())

    sns.histplot(x = col, hue = hue, data = df, ax = axes[0])
    sns.kdeplot(x = col, hue = hue, data = df, fill = True, ax = axes[1])
    sns.boxplot(y = col, hue = hue, data = df, x = [""] * len(df), ax = axes[2])
    sns.violinplot(y = col, hue = hue, data = df, x = [""] * len(df), ax = axes[3])
    
    fig.suptitle("For Feature:  " + col)
    axes[0].set_title("Histogram For Feature " + col)
    axes[1].set_title("KDE Plot For Feature " + col)   
    axes[2].set_title("Boxplot For Feature " + col)   
    axes[3].set_title("Violinplot For Feature " + col)            

In [ ]:
feature_dist_clas(mobile, "battery_power", "price_range")

In [ ]:
feature_dist_clas(titanic, "Fare", "Survived")

# 3) Categorical Variable vs Numerical Target

In [ ]:
def bar_box(df, col, target):
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 6), sharex = True)
    
    order = sorted(df[col].unique())
    
    sns.countplot(data = df, x = col, ax = axes[0], order = order)   
    sns.boxplot(data = df, x = col, ax = axes[1], y = target, order = order)
    
    fig.suptitle("For Feature:  " + col)
    axes[0].set_title("Countplot For " + col)
    axes[1].set_title(col + " --- " + target)
    
    for ax in fig.axes:
        plt.sca(ax)
        plt.xticks(rotation=90)

In [ ]:
bar_box(house_prices, "MSSubClass", "SalePrice")

In [ ]:
bar_box(car_price, "fueltype", "price")

# 4) Numerical Variable vs Numerical Target

In [ ]:
def plot_scatter(df, col, target):
    
    corr = df[[col, target]].corr()[col][1]    
    c = ["red"] if corr >= 0.7 else (["brown"] if corr >= 0.3 else\
                                    (["lightcoral"] if corr >= 0 else\
                                    (["blue"] if corr <= -0.7 else\
                                    (["royalblue"] if corr <= -0.3 else ["lightskyblue"]))))    

    fig, ax = plt.subplots(figsize = (6, 6))
    
    sns.scatterplot(x = col, y = target, data = df, c = c, ax = ax)        
    ax.set_title("Correlation between " + col + " and " + target + " is: " + str(corr.round(4)))

In [ ]:
plot_scatter(car_price, "horsepower", "price")

In [ ]:
plot_scatter(house_prices, "LotArea", "SalePrice")

In [ ]:
from scipy import stats

def feature_distribution(df, col, target):
    
    skewness = np.round(df[col].skew(), 3)
    kurtosis = np.round(df[col].kurtosis(), 3)


    fig, axes = plt.subplots(1, 3, figsize = (18, 6))

    sns.kdeplot(data = df, x = col, fill = True, ax = axes[0], color = "orangered")
    sns.boxplot(data = df, y = col, ax = axes[1], color = "orangered")
    stats.probplot(df[col], plot = axes[2])

    axes[0].set_title("Distribution \nSkewness: " + str(skewness) + "\nKurtosis: " + str(kurtosis))
    axes[1].set_title("Boxplot")
    axes[2].set_title("Probability Plot")
    fig.suptitle("For Feature:  " + col)

In [ ]:
feature_distribution(car_price, "horsepower", "price")

In [ ]:
feature_distribution(house_prices, "GrLivArea", "SalePrice")

In [ ]:
def heatmap(df):
    
    fig, ax = plt.subplots(figsize = (20, 20))
    
    sns.heatmap(df.corr(), cmap = "coolwarm", annot = True, fmt = ".2f", annot_kws = {"fontsize": 9},
                vmin = -1, vmax = 1, square = True, linewidths = 0.8, cbar = False)

In [ ]:
heatmap(car_price)

In [ ]:
heatmap(house_prices)